In [8]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras import backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from focal_loss import BinaryFocalLoss
from tensorflow.keras.optimizers.legacy import Adam


In [2]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=1,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.


Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [8]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use linear activation for Hinge loss
])


In [15]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
1399/1399 [==============================] - 54s 39ms/step - loss: 0.5225 - accuracy: 0.7405 - val_loss: 0.4246 - val_accuracy: 0.7465
Epoch 2/20
1399/1399 [==============================] - 52s 37ms/step - loss: 0.3967 - accuracy: 0.8127 - val_loss: 0.2942 - val_accuracy: 0.8507
Epoch 3/20
1399/1399 [==============================] - 53s 38ms/step - loss: 0.3062 - accuracy: 0.8628 - val_loss: 0.3521 - val_accuracy: 0.8472
Epoch 4/20
1399/1399 [==============================] - 54s 39ms/step - loss: 0.2569 - accuracy: 0.8863 - val_loss: 0.1986 - val_accuracy: 0.9062
Epoch 5/20
1399/1399 [==============================] - 55s 39ms/step - loss: 0.2254 - accuracy: 0.9049 - val_loss: 0.1896 - val_accuracy: 0.9097
Epoch 6/20
1399/1399 [==============================] - 55s 39ms/step - loss: 0.1908 - accuracy: 0.9149 - val_loss: 0.1824 - val_accuracy: 0.9410
Epoch 7/20
1399/1399 [==============================] - 55s 39ms/step - loss: 0.1641 - accuracy: 0.9328 - val_loss: 0.1170 -

In [17]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 76ms/step - loss: 0.0897 - accuracy: 0.9549
Test Loss: 0.08970168977975845, Test Accuracy: 0.9548611044883728


In [10]:
from tensorflow.keras.losses import BinaryFocalCrossentropy

In [10]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryFocalCrossentropy(), metrics=['accuracy'])


NameError: name 'BinaryFocalCrossentropy' is not defined

In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
1399/1399 [==============================] - 76s 54ms/step - loss: 0.1396 - accuracy: 0.7370 - val_loss: 0.1276 - val_accuracy: 0.7535
Epoch 2/20
1399/1399 [==============================] - 55s 39ms/step - loss: 0.1042 - accuracy: 0.8213 - val_loss: 0.0819 - val_accuracy: 0.8472
Epoch 3/20
1399/1399 [==============================] - 79s 56ms/step - loss: 0.0777 - accuracy: 0.8585 - val_loss: 0.0532 - val_accuracy: 0.8889
Epoch 4/20
1399/1399 [==============================] - 86s 61ms/step - loss: 0.0630 - accuracy: 0.8942 - val_loss: 0.0499 - val_accuracy: 0.9132
Epoch 5/20
1399/1399 [==============================] - 87s 62ms/step - loss: 0.0479 - accuracy: 0.9171 - val_loss: 0.0362 - val_accuracy: 0.9271
Epoch 6/20
1399/1399 [==============================] - 87s 62ms/step - loss: 0.0405 - accuracy: 0.9357 - val_loss: 0.0261 - val_accuracy: 0.9549
Epoch 7/20
1399/1399 [==============================] - 86s 62ms/step - loss: 0.0326 - accuracy: 0.9543 - val_loss: 0.0210 -

In [13]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 3s 282ms/step - loss: 0.0492 - accuracy: 0.9618
Test Loss: 0.049241337925195694, Test Accuracy: 0.9618055820465088


## Test CSL - dodanie niestandardowej macierzy wag

In [18]:
class_weights = {0: 1.0, # Waga dla klasy negatywnej
                 1: 100.0} 

In [19]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    class_weight = class_weights
)

Epoch 1/20
1399/1399 [==============================] - 52s 37ms/step - loss: 11.7525 - accuracy: 0.5861 - val_loss: 2.3241 - val_accuracy: 0.6806
Epoch 2/20
1399/1399 [==============================] - 53s 38ms/step - loss: 2.7768 - accuracy: 0.6848 - val_loss: 4.2395 - val_accuracy: 0.2951
Epoch 3/20
1399/1399 [==============================] - 54s 39ms/step - loss: 2.4826 - accuracy: 0.6805 - val_loss: 0.9758 - val_accuracy: 0.7431
Epoch 4/20
1399/1399 [==============================] - 55s 39ms/step - loss: 4.7902 - accuracy: 0.6112 - val_loss: 1.5368 - val_accuracy: 0.5069
Epoch 5/20
1399/1399 [==============================] - 55s 39ms/step - loss: 2.3593 - accuracy: 0.6226 - val_loss: 0.5859 - val_accuracy: 0.8646
Epoch 6/20
1399/1399 [==============================] - 56s 40ms/step - loss: 1.8827 - accuracy: 0.6969 - val_loss: 0.3701 - val_accuracy: 0.9062
Epoch 7/20
1399/1399 [==============================] - 56s 40ms/step - loss: 2.9231 - accuracy: 0.6898 - val_loss: 1.0959 

In [13]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 72ms/step - loss: 0.1223 - accuracy: 0.9514
Test Loss: 0.12229922413825989, Test Accuracy: 0.9513888955116272


# Wykorzystanie pretrenowanych sieci

### MobileNetV2

In [4]:
from tensorflow.keras.applications import MobileNetV2

In [5]:
img_height = img_width = 128

# Ładowanie MobileNetV2 bez górnej warstwy (top)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Zamrożenie wagi przetrenowanego modelu
base_model.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [7]:
# Dodanie warstw na wierzchu
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# To jest tylko przykład, można tutaj dostosować liczbę neuronów i warstw
output = Dense(1, activation='sigmoid')(x)  # Przykładowa warstwa wyjściowa dla klasyfikacji binarnej

In [9]:
# Skompletowanie nowego modelu
model = Model(inputs=base_model.input, outputs=output)

In [10]:
# Kompilacja modelu z niestandardową funkcją straty i optymalizatorem
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
# Wypisanie struktury modelu
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Conv1[0][0]']        

In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
1399/1399 [==============================] - 12s 8ms/step - loss: 0.1645 - accuracy: 0.9285 - val_loss: 0.0328 - val_accuracy: 0.9931
Epoch 2/20
1399/1399 [==============================] - 14s 10ms/step - loss: 0.0680 - accuracy: 0.9721 - val_loss: 0.0245 - val_accuracy: 0.9896
Epoch 3/20
1399/1399 [==============================] - 15s 10ms/step - loss: 0.0496 - accuracy: 0.9821 - val_loss: 0.0143 - val_accuracy: 1.0000
Epoch 4/20
1399/1399 [==============================] - 15s 11ms/step - loss: 0.0260 - accuracy: 0.9893 - val_loss: 0.0248 - val_accuracy: 0.9931
Epoch 5/20
1399/1399 [==============================] - 16s 12ms/step - loss: 0.0267 - accuracy: 0.9900 - val_loss: 0.0252 - val_accuracy: 0.9861
Epoch 6/20
1399/1399 [==============================] - 17s 12ms/step - loss: 0.0196 - accuracy: 0.9907 - val_loss: 0.0174 - val_accuracy: 0.9965
Epoch 7/20
1399/1399 [==============================] - 18s 13ms/step - loss: 0.0294 - accuracy: 0.9921 - val_loss: 0.0275 - 

In [13]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 82ms/step - loss: 0.0911 - accuracy: 0.9826
Test Loss: 0.09107708185911179, Test Accuracy: 0.9826388955116272
